In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(rc = {'figure.figsize':(15,8)})
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import random as rd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from statsmodels.tsa.vector_ar.var_model import VAR
from copy import deepcopy
from tqdm import tqdm
from matplotlib import pyplot
%matplotlib inline
import warnings

In [3]:
area_farmed = pd.read_csv('./../assets/area_farmed_june.csv')
land_cover = pd.read_csv('clc_land_.cover.csv')
fert = pd.read_csv('fertilizers.csv')
rain = pd.read_csv('rainfall.csv')
sun = pd.read_csv('sunshine.csv')
temp = pd.read_csv('temperature.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'clc_land_.cover.csv'

In [3]:
area_farmed
#2023-2021

,Statistic,Year,Type of Land Use,Region,UNIT,VALUE
0,Area Farmed in June,2013,Area farmed (AAU),State,000 Hectares,4477.8
1,Area Farmed in June,2013,Area farmed (AAU),Northern and Western,000 Hectares,1500.6
2,Area Farmed in June,2013,Area farmed (AAU),Border,000 Hectares,713.3
3,Area Farmed in June,2013,Area farmed (AAU),West,000 Hectares,787.3
4,Area Farmed in June,2013,Area farmed (AAU),Southern,000 Hectares,2045.2
...,...,...,...,...,...,...
2965,Area Farmed in June,2021,Rough grazing in use,South-East,000 Hectares,NaN
2966,Area Farmed in June,2021,Rough grazing in use,South-West,000 Hectares,NaN
2967,Area Farmed in June,2021,Rough grazing in use,Eastern and Midland,000 Hectares,NaN
2968,Area Farmed in June,2021,Rough grazing in use,Dublin and Mid-East,000 Hectares,NaN


In [19]:
area_farmer = area_farmed.rename({'Type of Land Use': 'Land_Type'})

In [4]:
land_cover
#2012-2018, not yearly but reffering to change withing the gap.

,Statistic,Year,State,UNIT,VALUE
0,Change from Non-irrigated arable land to Pastures,2012 - 2018,State,Area (hectares),58248
1,Change from Pastures to Non-irrigated arable land,2012 - 2018,State,Area (hectares),54111
2,Change from Pastures to Transitional woodland-...,2012 - 2018,State,Area (hectares),10428
3,Change from Land principally occupied by agric...,2012 - 2018,State,Area (hectares),2812
4,Change from Pastures to Construction sites,2012 - 2018,State,Area (hectares),1091
...,...,...,...,...,...
67,% Change from Construction sites to Transition...,2012 - 2018,State,%,<0.1
68,% Change from Land principally occupied by agr...,2012 - 2018,State,%,<0.1
69,% Change from Inland marshes to Non-irrigated ...,2012 - 2018,State,%,<0.1
70,% Change from Sport and leisure facilities to ...,2012 - 2018,State,%,<0.1


In [5]:
fert
#1980 - 2021, monthly.

,Statistic,Month,Type of Fertiliser,UNIT,VALUE
0,Fertiliser Price,1980M01,Calcium Ammonium Nitrate (27.5% N),Euro per Tonne,121
1,Fertiliser Price,1980M01,Urea (46% N),Euro per Tonne,144
2,Fertiliser Price,1980M01,Granular Superphosphate (16% P),Euro per Tonne,160
3,Fertiliser Price,1980M01,Muriate of Potash (50% K),Euro per Tonne,109
4,Fertiliser Price,1980M01,Compound 0-10-20,Euro per Tonne,130
...,...,...,...,...,...
12043,Fertiliser Price,2021M10,Compound 15-3-20,Euro per Tonne,.
12044,Fertiliser Price,2021M10,Compound 24-2.2-10,Euro per Tonne,.
12045,Fertiliser Price,2021M10,Compound 24-2.2-4.5,Euro per Tonne,423
12046,Fertiliser Price,2021M10,Compound 19-0-15,Euro per Tonne,501


In [18]:
fert = fert.rename(columns={"Type of Fertiliser": "Fertilizer_Type"})

In [6]:
rain

,Statistic,Month,Meteorological Weather Station,UNIT,VALUE
0,Total Rainfall,1958M01,Belmullet,Millimetres,160.2
1,Total Rainfall,1958M01,Birr,Millimetres,95.6
2,Total Rainfall,1958M01,Valentia Observatory,Millimetres,214.8
3,Total Rainfall,1958M01,Casement,Millimetres,NaN
4,Total Rainfall,1958M01,Claremorris,Millimetres,146.9
...,...,...,...,...,...
34510,Raindays (0.2mm or More),2021M11,Malin head,Number,26.0
34511,Raindays (0.2mm or More),2021M11,Mullingar,Number,23.0
34512,Raindays (0.2mm or More),2021M11,Roches Point,Number,14.0
34513,Raindays (0.2mm or More),2021M11,Rosslare,Number,NaN


In [17]:
rain = rain.rename(columns={"Meteorological Weather Station": "Met_Weather_Station"})

In [7]:
sun

,Statistic,Month,Meteorological Weather Station,UNIT,VALUE
0,Total Sunshine Hours,1958M01,Belmullet,Number,43.7
1,Total Sunshine Hours,1958M01,Birr,Number,64.6
2,Total Sunshine Hours,1958M01,Valentia Observatory,Number,46.8
3,Total Sunshine Hours,1958M01,Casement,Number,NaN
4,Total Sunshine Hours,1958M01,Claremorris,Number,51.6
...,...,...,...,...,...
23005,Most Sunshine Hours in a Day,2021M11,Malin head,Number,NaN
23006,Most Sunshine Hours in a Day,2021M11,Mullingar,Number,NaN
23007,Most Sunshine Hours in a Day,2021M11,Roches Point,Number,NaN
23008,Most Sunshine Hours in a Day,2021M11,Rosslare,Number,NaN


In [15]:
sun = sun.rename(columns={"Meteorological Weather Station": "Met_Weather_Station"})

In [9]:
temp

,Statistic,Month,Meteorological Weather Station,UNIT,VALUE
0,Average Maximum Temperature,1958M01,Belmullet,Degrees C,8.2
1,Average Maximum Temperature,1958M01,Birr,Degrees C,7.8
2,Average Maximum Temperature,1958M01,Valentia Observatory,Degrees C,9.7
3,Average Maximum Temperature,1958M01,Casement,Degrees C,NaN
4,Average Maximum Temperature,1958M01,Claremorris,Degrees C,7.4
...,...,...,...,...,...
57520,Lowest Temperature,2021M11,Malin head,Degrees C,3.7
57521,Lowest Temperature,2021M11,Mullingar,Degrees C,0.2
57522,Lowest Temperature,2021M11,Roches Point,Degrees C,1.8
57523,Lowest Temperature,2021M11,Rosslare,Degrees C,NaN


In [13]:
temp = temp.rename(columns={"Meteorological Weather Station": "Met_Weather_Station"})

In [10]:
temp.Statistic.unique()

array(['Average Maximum Temperature', 'Average Minimum Temperature',
       'Mean Temperature', 'Highest Temperature', 'Lowest Temperature'],
      dtype=object)

In [14]:
temp.Met_Weather_Station.unique()

array(['Belmullet', 'Birr', 'Valentia Observatory', 'Casement',
       'Claremorris', 'Clones', 'Cork airport', 'Dublin airport',
       'Galway', 'Kilkenny', 'Malin head', 'Mullingar', 'Roches Point',
       'Rosslare', 'Shannon airport'], dtype=object)